##Taste of Supervised Learning

# Naive-Bayes Classification
## Definition

Naive Bayes is a relatively simple classification algorithm based on probability and uses Bayes Theorom with an independence assumption among the features in the data. The fundamental idea of Naive Bayes is that it computes the probability of every class, which we want to reveal, based on the probability of every feature in the data.

According to Naive Bayes algorithm, we are going to assume that every feature in the data is in an independent condition on the outcome probability of each separate class. Let's assume that we are doing a car classification and we have a data such as;

| buying   | maint    | doors    | persons  | lug-boot | safety   | class    |
| :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| vvhigh   | vhigh    | 2        | 2        | small    | low      | unacc    |

**Description of dataset:**
* CAR                      car acceptability
    * PRICE                  overall price
        * _buying_               buying price
        * _maint_                price of the maintenance
* TECH                   technical characteristics
    * COMFORT              comfort
        * _doors_              number of doors
        * _persons_            capacity in terms of persons to carry
        * _lug-boot_           the size of luggage boot
    * _safety_               estimated safety of the car
   
Naive Bayes assumes that features above occur independently of each other.

In machine learning, Naive Bayes is advantageous against other commonly used classification algorithms because of its simplicity, speed and accuracy on small datasets and it also enables us to make classification despite missing information. Naive Bayes is a supervised learning algorithm because it needs to be trained with a labeled dataset.

## Bayes Theorem
Consider two events, $A$ and $B$. For example, $A$ is a set of car features, which are $A \in \{ vvhigh, vhigh, 2, 2, small, low \}$,and $B$ is a set of car classes that are $B \in \{ unacc, acc, good, vgood \}$

<img src="vennDiagramOfBayesTheorem.png"  style="height:50%;width:70%" />

* $A \cap B$ means the intersection of $A$ and $B$.
* $P(A \mid B)$ is read as probability of A given B.

When we know that $B$ is given (Event $B$ has occurred), it means our sample space is $B$ that is the right figure. Now we are trying to compute the probability of also occuring $A$ at the same time (the conditional probability of $A$). It is obvious that we are trying to find the probability of $A \cap B$ given that we are in the space of $B$.

\begin{equation}
P(A \mid B) = \frac{P(A \cap B)}{P(B)}
\end{equation}

We can rewrite $P(A \cap B)$ as $P(A, B)$. Two of these mean the probability of $A$ and $B$ at the same time. So the new form of the equation is :

\begin{equation}
P(A \mid B) = \frac{P(A, B)}{P(B)}
\end{equation}

For the probability of $A$ and $B$, we can deduce equations below from the figure above.

\begin{align}
& P(A, B) = P(B, A) = P(A \mid B)P(B) \\
& P(A, B) = P(B, A) = P(B \mid A)P(A)
\end{align}

Let's look at the new form of the equation putting the second form of $P(A, B)$:

\begin{equation}
P(A \mid B) = \frac{P(B \mid A)P(A)}{P(B)}
\end{equation}

This equation is known as **Bayes Theorem**.
* $P(A \mid B)$ : posterior that is the probability of $A$ when it is known that $B$ is given
* $P(B)$ : evidence that is the marginal probability of $B$
* $P(B \mid A)$ : likelihood
* $P(A)$ : prior probability that is marginal probabiliy of $A$

## Naive-Bayes Formulation
Suppose we have a dataset which each observation belongs to a class from the finite set $C = \{ c_1, c_2, ..., c_n \}$ and each observation constitutes from a few features $F = \{ f_1, f_2, ..., f_b \}$. If we could compute the probabilities of $P(c_1 | F), P(c_2 | F), ..., P(c_n | F)$ then we could predict the class for a new observation $i$ to be one of those which have the highest probability.

To compute the conditional probabilities, we can use Bayes Theorem;

\begin{equation}
P(c_i \mid f_1, f_2, \dots ,f_b) = \frac{P(f_1, f_2, \dots ,f_b \mid c_i)P(c_i)}{P(f_1, f_2, \dots ,f_b)}
\end{equation}

As you know, Naive-Bayes supposes that all features are in independent conditions, therefore we can rewrite this equation like;

\begin{equation}
P(c_i \mid f_1, f_2, \dots ,f_b) = \frac{P(f_1 \mid c_i)P(f_2 \mid c_i) \dots P(f_b \mid c_i)P(c_i)}{P(f_1, f_2, \dots ,f_b)}
\end{equation}

The final form of equation is

\begin{align}
& \text{for} \; i = 1, 2, \dots , n \\
& P(c_i \mid f_1, f_2, \dots ,f_b) = P(c_i) \frac{\Pi_{j=1}^b P(f_j \mid c_i)}{P(f_1, f_2, \dots ,f_b)}
\end{align}

Since $P(f_1, f_2, \dots ,f_b)$ is a constant, we can use the classification rule below.

\begin{align}
& P(c_i \mid f_1, f_2, \dots ,f_b) \propto P(c_i) \Pi_{j=1}^b P(f_j \mid c_i)
\end{align}

When a set of features came, we are going to compute the possibilities of each class for the feature set by using the equation above.

Let's look at how we can code it.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
# for data manipulation
import pandas as pd
# for array operations
import numpy as np

# read dataset from csv file
dataset = pd.read_csv("/content/gdrive/MyDrive/2023_Mobility/Talks/ToSuLe-main/datasets/car-eval.csv")
#print(dataset)
dataset.describe()

,buying,maint,doors,persons,lug_boot,safety,clazz
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


In [3]:
# if dataset.index % test_indis == 0
# then it is going to be used as test dataset
# they will not be attended into the train dataset
test_indis = 79

test_dataset = dataset[dataset.index % test_indis == 0]
train_dataset = dataset[dataset.index % test_indis != 0]

# total count of sample space
total = len(train_dataset)

Let's look at the probabilities of each item in the data one by one.

In [7]:
p_clazz[index_clazz]

array([0.03692849])

In [6]:
# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (buying X clazz)
p_clazz = np.zeros((len(keys_clazz)))

# zip only buying and clazz values
for u in train_dataset["clazz"]:
    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u)

    # increment 1 the count of the intersection of buying x clazz pair.
    p_clazz[index_clazz] += 1

# let's normalize the possibilities
p_clazz = p_clazz / np.sum(p_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_clazz = pd.DataFrame(p_clazz, keys_clazz)

# possibilities of class values
df_p_clazz

,0
unacc,0.700469
acc,0.222743
vgood,0.036928
good,0.039859


In [8]:
# unique values of buying column
keys_buying = np.array(train_dataset["buying"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (buying X clazz)
p_buying_given_clazz = np.zeros((len(keys_buying), len(keys_clazz)))

# zip only buying and clazz values
for u in zip(train_dataset["buying"], train_dataset["clazz"]):
    # get index of current buying value
    index_buying = np.where(keys_buying == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of buying x clazz pair.
    p_buying_given_clazz[[index_buying], [index_clazz]] += 1

# we create a pandas dataframe to visualize the table more familiar
df_p_buying_given_clazz = pd.DataFrame(p_buying_given_clazz, keys_buying, keys_clazz)

# counts of buying given class
df_p_buying_given_clazz

,unacc,acc,vgood,good
vhigh,355.0,71.0,0.0,0.0
high,320.0,107.0,0.0,0.0
med,264.0,115.0,25.0,22.0
low,256.0,87.0,38.0,46.0


In [9]:
# let's normalize the possibilities
p_buying_given_clazz = p_buying_given_clazz / np.sum(p_buying_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_buying_given_clazz = pd.DataFrame(p_buying_given_clazz, keys_buying, keys_clazz)

# possibilities of buying values given class
df_p_buying_given_clazz

,unacc,acc,vgood,good
vhigh,0.297071,0.186842,0.000000,0.000000
high,0.267782,0.281579,0.000000,0.000000
med,0.220921,0.302632,0.396825,0.323529
low,0.214226,0.228947,0.603175,0.676471


In [10]:
# unique values of maint column
keys_maint = np.array(train_dataset["maint"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (maint X clazz)
p_maint_given_clazz = np.zeros((len(keys_maint), len(keys_clazz)))

# zip only maint and clazz values
for u in zip(train_dataset["maint"], train_dataset["clazz"]):
    # get index of current maint value
    index_maint = np.where(keys_maint == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of maint x clazz pair.
    p_maint_given_clazz[[index_maint], [index_clazz]] += 1

# let's normalize the possibilities
p_maint_given_clazz = p_maint_given_clazz / np.sum(p_maint_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_maint_given_clazz = pd.DataFrame(p_maint_given_clazz, keys_maint, keys_clazz)

# possibilities of maint values given class
df_p_maint_given_clazz

,unacc,acc,vgood,good
vhigh,0.297071,0.186842,0.000000,0.000000
high,0.259414,0.271053,0.206349,0.000000
med,0.220921,0.302632,0.380952,0.338235
low,0.222594,0.239474,0.412698,0.661765


In [11]:
# unique values of doors column
keys_doors = np.array(train_dataset["doors"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (doors X clazz)
p_doors_given_clazz = np.zeros((len(keys_doors), len(keys_clazz)))

# zip only doors and clazz values
for u in zip(train_dataset["doors"], train_dataset["clazz"]):
    # get index of current doors value
    index_doors = np.where(keys_doors == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of doors x clazz pair.
    p_doors_given_clazz[[index_doors], [index_clazz]] += 1

# let's normalize the possibilities
p_doors_given_clazz = p_doors_given_clazz / np.sum(p_doors_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_doors_given_clazz = pd.DataFrame(p_doors_given_clazz, keys_doors, keys_clazz)

# possibilities of doors values given class
df_p_doors_given_clazz

,unacc,acc,vgood,good
2,0.268619,0.213158,0.142857,0.220588
3,0.247699,0.257895,0.238095,0.264706
4,0.241841,0.265789,0.301587,0.250000
5more,0.241841,0.263158,0.317460,0.264706


In [12]:
# unique values of persons column
keys_persons = np.array(train_dataset["persons"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (persons X clazz)
p_persons_given_clazz = np.zeros((len(keys_persons), len(keys_clazz)))

# zip only persons and clazz values
for u in zip(train_dataset["persons"], train_dataset["clazz"]):
    # get index of current persons value
    index_persons = np.where(keys_persons == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of persons x clazz pair.
    p_persons_given_clazz[[index_persons], [index_clazz]] += 1

# let's normalize the possibilities
p_persons_given_clazz = p_persons_given_clazz / np.sum(p_persons_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_persons_given_clazz = pd.DataFrame(p_persons_given_clazz, keys_persons, keys_clazz)

# possibilities of persons values given class
df_p_persons_given_clazz

,unacc,acc,vgood,good
2,0.477824,0.000000,0.000000,0.000000
4,0.257741,0.513158,0.460317,0.529412
more,0.264435,0.486842,0.539683,0.470588


In [13]:
# unique values of lugboot column
keys_lugboot = np.array(train_dataset["lug_boot"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (lugboot X clazz)
p_lugboot_given_clazz = np.zeros((len(keys_lugboot), len(keys_clazz)))

# zip only lugboot and clazz values
for u in zip(train_dataset["lug_boot"], train_dataset["clazz"]):
    # get index of current lugboot value
    index_lugboot = np.where(keys_lugboot == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of lugboot x clazz pair.
    p_lugboot_given_clazz[[index_lugboot], [index_clazz]] += 1

# let's normalize the possibilities
p_lugboot_given_clazz = p_lugboot_given_clazz / np.sum(p_lugboot_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_lugboot_given_clazz = pd.DataFrame(p_lugboot_given_clazz, keys_lugboot, keys_clazz)

# possibilities of lugboot values given class
df_p_lugboot_given_clazz

,unacc,acc,vgood,good
small,0.372385,0.271053,0.000000,0.308824
med,0.323013,0.355263,0.380952,0.338235
big,0.304603,0.373684,0.619048,0.352941


In [14]:
# unique values of safety column
keys_safety = np.array(train_dataset["safety"].unique())

# unique values of clazz column
keys_clazz = np.array(train_dataset["clazz"].unique())

# let's initiate a matrix to hold counts (safety X clazz)
p_safety_given_clazz = np.zeros((len(keys_safety), len(keys_clazz)))

# zip only safety and clazz values
for u in zip(train_dataset["safety"], train_dataset["clazz"]):
    # get index of current safety value
    index_safety = np.where(keys_safety == u[0])

    # get index of current clazz value
    index_clazz = np.where(keys_clazz == u[1])

    # increment 1 the count of the intersection of safety x clazz pair.
    p_safety_given_clazz[[index_safety], [index_clazz]] += 1

# let's normalize the possibilities
p_safety_given_clazz = p_safety_given_clazz / np.sum(p_safety_given_clazz, axis=0, keepdims=True)

# we create a pandas dataframe to visualize the table more familiar
df_p_safety_given_clazz = pd.DataFrame(p_safety_given_clazz, keys_safety, keys_clazz)

# possibilities of safety values given class
df_p_safety_given_clazz

,unacc,acc,vgood,good
med,0.294561,0.471053,0.0,0.558824
high,0.230126,0.528947,1.0,0.441176
low,0.475314,0.000000,0.0,0.000000


In [15]:
# let's compute marginal probabilities of buying values
p_buying = np.sum(p_clazz * p_buying_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_buying = pd.DataFrame(p_buying, keys_buying)

# possibilities of buying values
df_p_buying

,0
vhigh,0.249707
high,0.250293
med,0.249707
low,0.250293


In [16]:
# let's compute marginal probabilities of maint values
p_maint = np.sum(p_clazz * p_maint_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_maint = pd.DataFrame(p_maint, keys_maint)

# possibilities of maint values
df_p_maint

,0
vhigh,0.249707
high,0.249707
med,0.249707
low,0.250879


In [17]:
# let's compute marginal probabilities of doors values
p_doors = np.sum(p_clazz * p_doors_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_doors = pd.DataFrame(p_doors, keys_doors)

# possibilities of doors values
df_p_doors

,0
2,0.249707
3,0.250293
4,0.249707
5more,0.250293


In [18]:
# let's compute marginal probabilities of persons values
p_persons = np.sum(p_clazz * p_persons_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_persons = pd.DataFrame(p_persons, keys_persons)

# possibilities of persons values
df_p_persons

,0
2,0.334701
4,0.332943
more,0.332356


In [19]:
# let's compute marginal probabilities of lug_boot values
p_lugboot = np.sum(p_clazz * p_lugboot_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_lugboot = pd.DataFrame(p_lugboot, keys_lugboot)

# possibilities of lug_boot values
df_p_lugboot

,0
small,0.333529
med,0.332943
big,0.333529


In [20]:
# let's compute marginal probabilities of safety values
p_safety = np.sum(p_clazz * p_safety_given_clazz, axis=1)

# we create a pandas dataframe to visualize the table more familiar
df_p_safety = pd.DataFrame(p_safety, keys_safety)

# possibilities of safety values
df_p_safety

,0
med,0.333529
high,0.333529
low,0.332943


In [21]:
correct = 0
for idx, item in test_dataset.iterrows():
    alpha = 1
    denominator = 0
    possibilities = {}
    for cls in keys_clazz:
        p = 1
        p *= (df_p_buying_given_clazz[cls][item.buying])
        p *= (df_p_maint_given_clazz[cls][item.maint])
        p *= (df_p_doors_given_clazz[cls][item.doors])
        p *= (df_p_persons_given_clazz[cls][item.persons])
        p *= (df_p_lugboot_given_clazz[cls][item.lug_boot])
        p *= (df_p_safety_given_clazz[cls][item.safety])
        p *= df_p_clazz[0][cls]

        denominator += p

        possibilities[cls] = p
    # end of for loop

    possibilities.update({k: v / denominator for k, v in possibilities.items()})

    #print(possibilities)
    correct += max(possibilities, key=possibilities.get) == item.clazz
# end of for loop

print("accuracy: %d%%" % ((correct / len(test_dataset)) * 100))

accuracy: 81%


## References
* UCL Machine Learning Repository, https://archive.ics.uci.edu/ml/index.php (22/10/2017)
* Scikit-learn, Naive Bayes, http://scikit-learn.org/stable/modules/naive_bayes.html (22/10/2017)